In [62]:
from PIL import Image
import torch
import torch.utils.data
import torchvision
import numpy as np
import matplotlib
import os

In [64]:
def img_seg(img,label):
    hight,width = img.size
    w = 32
    id = 0
    j = 6
    new_imgs = []
    while(j + w <= width):
        i = 14
        while(i +w <= hight):
            new_imgs.append((img.crop((i, j, i + w, j + w)).convert('RGB'),label))
            id += 1
            i += w
        j += w
#    print(hight,width,id)
    return new_imgs

In [65]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, root, datatxt, datatype,transform = None, target_transform = None):
        super(MyDataSet,self).__init__()
        fh = open(root + datatxt, 'r')
        imgs = []
       # j = 0
        for line in fh:
            columns = line.split('|')
            if columns[3].replace("\n","") == datatype:
                imgname = columns[0]
                imglb = imgname[4]
                if imglb == 'B':
                    imglb = 0
                else:
                    imglb = 1
                lb = []
                mag = columns[1]  # 40, 100, 200, or 400
                fold = columns[2]
                img = Image.open((root + '%s/' + '%sX/' + imgname )% (datatype,mag)).convert('RGB')
                new_img = img_seg(img,imglb)
                imgs.extend(new_img)
        #        j = j+1
        #        if imglb == 'M':
        #            print(j,new_labels,'\n')
         
                
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        
    def __getitem__(self,index):
        img,target = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        return img,target
    
    def __len__(self):
        return len(self.imgs)

In [66]:
root = './fold2/'
fold2_train_data = MyDataSet(root = root,datatxt = 'dsfold2.txt',datatype = 'train',transform = torchvision.transforms.ToTensor())
fold2_test_data = MyDataSet(root = root,datatxt = 'dsfold2.txt',datatype = 'test',transform = torchvision.transforms.ToTensor())

In [67]:
print(fold2_train_data[0],'\n') #first sample
print(fold2_train_data[0][0],'\n') #pixels of first sample
print(fold2_train_data[0][1],'\n') #label of first sample
print(fold2_train_data[0][0][1]) #first channel of pixels of first sample(RGB have three channels)

(tensor([[[0.7843, 0.7725, 0.7608,  ..., 0.7569, 0.7569, 0.7647],
         [0.7686, 0.7686, 0.7569,  ..., 0.8118, 0.8157, 0.8039],
         [0.7725, 0.7647, 0.7569,  ..., 0.8196, 0.8078, 0.7922],
         ...,
         [0.7412, 0.7686, 0.7765,  ..., 0.7176, 0.7216, 0.7216],
         [0.7412, 0.7686, 0.7647,  ..., 0.7137, 0.7098, 0.7216],
         [0.7451, 0.7490, 0.7373,  ..., 0.7294, 0.7294, 0.7529]],

        [[0.5725, 0.5725, 0.5804,  ..., 0.7020, 0.7020, 0.7020],
         [0.5686, 0.5765, 0.5804,  ..., 0.7569, 0.7608, 0.7412],
         [0.5804, 0.5804, 0.5882,  ..., 0.7608, 0.7529, 0.7373],
         ...,
         [0.6549, 0.6863, 0.7020,  ..., 0.6314, 0.6392, 0.6314],
         [0.6549, 0.6863, 0.6941,  ..., 0.6392, 0.6353, 0.6471],
         [0.6706, 0.6667, 0.6667,  ..., 0.6627, 0.6627, 0.6863]],

        [[0.7059, 0.6941, 0.6941,  ..., 0.7608, 0.7608, 0.7529],
         [0.6980, 0.7020, 0.6941,  ..., 0.8235, 0.8275, 0.7922],
         [0.7059, 0.7059, 0.7059,  ..., 0.8353, 0.8196, 0

In [68]:
a =len(fold2_train_data[0][0][0])
b =len(fold2_train_data[0][0])
c =len(fold2_train_data)
d =len(fold2_train_data[0])
print(a,b,d,c)

32 3 2 1618764


In [69]:
kwarg = {'num_workers': 1,'pin_memory': True}
trainloader2 = torch.utils.data.DataLoader(dataset = fold2_train_data,batch_size=64,shuffle = True, **kwarg)
testloader2 = torch.utils.data.DataLoader(dataset = fold2_test_data,batch_size=64,shuffle = True, **kwarg)